In [1]:
from itertools import product
import os

In [2]:
def minisat_solve(problem_name, cnf):
    
    with open(f"{problem_name}.cnf", "w") as inp:
        inp.write(cnf.dimacs())
    
    os.system(f"minisat {problem_name}.cnf {problem_name}_res.cnf")
    
    with open(f"{problem_name}_res.cnf", "r") as res_file:
        res = res_file.readlines()
       
    sat = res[0]
    val = res[1]
    if sat == "SAT\n":
        print("SAT")
        val = val.split(" ")
        for lit in val[:-1]:
            if lit[0] != '-':
                print(f"{cnf.num_to_var[int(lit)]}")
    else:
        print("UNSAT")

In [3]:
class CNF:
    def __init__(self):
        self.clauses = []
        self.var_to_num = {}
        self.num_to_var = {}
    
    def add_clause(self, clause):
        for lit in clause:
            var_name = lit.strip('-')
            if var_name not in self.var_to_num:
                var_num = len(self.var_to_num) + 1
                self.var_to_num[var_name] = var_num
                self.num_to_var[var_num] = var_name
                
        self.clauses.append(clause)
        
    def dimacs(self):
        res = f"p cnf {len(self.clauses)} {len(self.var_to_num)}\n"
        for cls in self.clauses:
            for lit in cls:
                var_name = lit.strip('-')
                if lit[0] == '-':
                    res += f"-{self.var_to_num[var_name]} "
                else:
                    res += f"{self.var_to_num[var_name]} "
            res += "0\n"
        return res

In [4]:
def n_dame(n):
    cnf = CNF()
    
    # U svakom redu treba postaviti po jednu damu
    for i in range(n):
        clause = [f'p{i}{j}' for j in range(n)]
        cnf.add_clause(clause)
        
    # U svakom redu moze biti najvise jedna dama
    for k in range(n):
        for i in range(n - 1):
            for j in range(i + 1, n):
                cnf.add_clause([f"-p{k}{i}", f"-p{k}{j}"])
                
    # U svakoj kolini moze biti najvise jedna dama
    for k in range(n):
        for i in range(n - 1):
            for j in range(i + 1, n):
                cnf.add_clause([f"-p{i}{k}", f"-p{j}{k}"])
                
    
        
    for i, j, k, l in product(range(n), repeat=4):
        if k > i and abs(k - i) == abs(l - j):
            cnf.add_clause([f"-p{i}{j}", f"-p{k}{l}"])
            
    minisat_solve(f"{n}_dame", cnf)

In [5]:
n_dame(4)

============================[ Problem Statistics ]=============================
|                                                                             |
|  Number of variables:            16                                         |
|  Number of clauses:              80                                         |
|  Parse time:                   0.00 s                                       |
|  Eliminated vars:                 4                                           |
|  Vars set       :                 0                                           |
|  Eliminated clauses:           0.00 Mb                                      |
|  Simplification time:          0.00 s                                       |
|                                                                             |
============================[ Search Statistics ]==============================
| Conflicts |          ORIGINAL         |          LEARNT          | Progress |
|           |    Vars  Clauses Liter